# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
df = pd.read_csv("insurance.csv",sep=",")# Загрузка данных в датафрейм

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменам пола и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
# Создание и обучение модели

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Замена региона на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
# Формирование признаков  целевой переменной.

In [ ]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X = df [[ 'age',	'sex',	'bmi',	'children',	'smoker', 'region_northeast',	'region_northwest',	'region_southeast',	'region_southwest']]
y =df [ 'charges']

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

## Создаем и обучаем модель линейной регресии

In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)
lr.coef_

array([  259.14592221,   -57.65711037,   332.75473657,   511.16429173,
       24337.55251101,   734.36299722,   305.41184119,  -668.46532002,
        -371.30951839])

In [ ]:
# Визуальная оценка того что получилось и что предсказывает модель
pd.DataFrame([y_test.values, lr.predict(X_test)]).T


,0,1
0,20462.99766,14784.208089
1,24873.38490,35383.528237
2,5966.88740,5950.090436
3,4571.41305,5259.916350
4,14394.39815,14648.484205
...,...,...
263,16776.30405,28519.812602
264,5377.45780,8861.738796
265,16069.08475,16451.751476
266,9222.40260,11240.161931


In [ ]:
#  Результат
lr.predict(X_test)

array([14784.20808882, 35383.5282371 ,  5950.09043629,  5259.91634961,
       14648.48420495, 39696.33207717, 11813.95650463, 12655.24596894,
        5733.56670686,  3324.9425058 ,  5824.87544032,  1487.53331241,
       15270.19678865,  9996.42655337,  9959.3876009 , 16217.5754588 ,
        9430.67097836,  9977.44439235,  5935.85713714, 15523.20770782,
        2142.53785933,  7639.21471501,  7739.66340661,  5631.37128574,
        4445.3147364 , 35057.8296338 ,  6100.57751618,  2844.93664973,
       13096.42355995,  4217.21095742, 10515.71852217, 14867.4664586 ,
       13683.89002169,   573.39611348, 35828.04499436, 12484.91559276,
       15401.66018084, 12313.1412939 ,  3246.57289349,  5752.84881647,
        7052.39425655,  3015.16081378,  5202.4969079 , 25695.51001761,
       12823.94133372,  2882.37870617,  8640.38501778, 36081.27538239,
       15156.21218698,  6760.29721619,  5433.88497631,  6559.36469089,
        2805.24206922,  7684.45213362, 37466.68162675, 38218.24312243,
      

In [ ]:
# Оценка качества метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, lr.predict(X_test))

38104059.95484449

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

36196396.10083649

## Получаем предсказание и оцениваем качество

## Делаем предсказание для одного человека

In [ ]:
# Заполнение данных по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([27319.12058818])